In [ ]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
import time as time

# what follows is from the OpenMM script builder
# with some minor modifications...

## Simulation setup

In [ ]:
# read in a starting structure for ethane and the
# corresponding force field file
pdb = app.PDBFile('ethane.pdb')
forcefield = app.ForceField('ethane.gaff2.xml')

# setup system by taking topology from pdb file;
# run gas phase simulation with 2 fs time step (using SHAKE)
# at 298.15 K using a Langevin thermostat (integrator) with
# coupling constant of 5.0 ps^-1
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, 
                                 constraints=app.HBonds)
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 
                                   2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)

platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

## Energy minimization

This reduces the potential energy of the system before beginning dynamics. This eliminates "bad" (i.e., overly close) contacts and generally leads to more stable simulation behavior.

In [ ]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy before minimization is %s" % st.getPotentialEnergy())

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print("Potential energy after minimization is %s" % st.getPotentialEnergy())

## Equilibration

We will run a (very) short equilibration simulation to bring the molecule up to our desired temperature.  If this were a periodic system, we would also aim to bring the density/volume to equilibrium at the desired pressure.

In [ ]:
print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(2500)

## Production

Now we run a long MD simulation with parameters that are identical to the equilibration phase (other than simulation length, of course!).  We will also save a trajectory file (i.e., corodinats vs. time) of this simulation that we can analyze afterward using MDTraj (or other trajectory analysis tools).

In [ ]:
print('Running Production...')

tinit=time.time()
simulation.reporters.clear()
# output basic simulation information below every 250000 steps/500 ps
simulation.reporters.append(app.StateDataReporter(stdout, 250000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))
# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps/0.2 ps
simulation.reporters.append(app.DCDReporter('ethane_sim.dcd', 100))

# run the simulation for 1.0x10^7 steps/20 ns
simulation.step(10000000)
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')